In [1]:
import random
import numpy as np
import math

In [2]:
class Tree:
    def __init__(self, max_depth):
        self.max_depth = max_depth
        self.n_nodes = 0
        self.root = None
        self.fitness = float('inf')
        
        
    def copy(self):
        tree = Tree(self.max_depth)
        tree.n_nodes = self.n_nodes
        tree.root = self.root.copy()
        tree.fitness = self.fitness
        return tree
    
    
    def calculate_n_nodes(self):
        self.n_nodes = self.calculate_n_nodes_rec(self.root)
    
    
    def calculate_n_nodes_rec(self, node):
        current = 1
        if node.left != None:
            current += self.calculate_n_nodes_rec(node.left)
        if node.right != None:
            current += self.calculate_n_nodes_rec(node.right)
            
        return current
        
        
    def calculate_fitness(self, data, labels, mean_Y):
        sum_error = 0
        sum_mean = 0
        for i in range(data.shape[0]):
            sum_error += (labels[i]-self.evaluate(data[i]))**2
            sum_mean += (labels[i]-mean_Y)**2
            
        self.fitness = math.sqrt(sum_error/sum_mean)
        return self.fitness
        
        
    def select_node(self, index):
        node = self.select_rec(index, self.root, 0)
        return (node[1], node[2], node[3])
        
        
    def select_rec(self, index, node, current):
        if index == current:
            return (index, node, None, None)
        else:
            if node.left != None:
                l_index, l_node, l_parent, l_direction = self.select_rec(index, node.left, current+1)
                if l_index == index:
                    if l_parent == None:
                        return (l_index, l_node, node, 'left')
                    else:
                        return (l_index, l_node, l_parent, l_direction)
                current = l_index
            if node.right != None:
                r_index, r_node, r_parent, r_direction = self.select_rec(index, node.right, current+1)
                if r_index == index:
                    if r_parent == None:
                        return (r_index, r_node, node, 'right')
                    else:
                        return (r_index, r_node, r_parent, r_direction)
                current = r_index
            return (current, None, None, None)
    
        
    def evaluate(self, variables):
        return self.evaluate_rec(variables, self.root)
    
    
    def evaluate_rec(self, variables, node):
        if node.node_type == 'terminal':
            if type(node.value) is float:
                return node.value
            else:
                pos = int(node.value[1:])
                return variables[pos]
        else:
            left = self.evaluate_rec(variables, node.left)
            right = self.evaluate_rec(variables, node.right)
            if node.value == '+':
                return left + right
            elif node.value == '-':
                return left - right
            elif node.value == '*':
                return left * right
            else:
                if right != 0:
                    return left / right
                else:
                    return float('inf')
        
        
    def __str__(self):
        return self.print_tree(self.root)
    
    
    def print_tree(self, node):
        string = str(node)
        if node.left != None:
            string += " " + self.print_tree(node.left)
            
        if node.right != None:
            string += " " + self.print_tree(node.right)
            
        return string
        
        
        
class Node:  
    def __init__(self, node_type, nvars, value=None, left=None, right=None):
        self.node_type = node_type
        self.left = left
        self.right = right
        self.value = value
        self.nvars = nvars
        if value == None:
            if node_type == 'function':
                self.value = random.choice(['+', '-', '*', '/'])
            elif node_type == 'terminal':
                choice = random.choice(['x'+str(i) for i in range(nvars)] + ['const'])
                if choice == 'const':
                    self.value = random.random()
                    if random.randint(0,1) == 1:
                        self.value *= -1
                else:
                    self.value = choice
                
                
    def copy(self):
        left = None
        right = None
        if self.left != None:
            left = self.left.copy()
        if self.right != None:
            right = self.right.copy()
        return Node(self.node_type, self.nvars, self.value, left, right)
                
                
    def __str__(self):
        return str(self.value)

In [3]:
class GeneticProgramming:
    def __init__(self, pop_size, ngenerations, max_depth, tournament, pc, pm, elitism, nvars, train_data, train_labels, test_data, test_labels):
        self.pop_size = pop_size
        self.ngenerations = ngenerations
        self.max_depth = max_depth
        self.tournament = tournament
        self.pc = pc
        self.pm = pm
        self.elistism = elitism
        self.nvars = nvars
        self.train_data = train_data
        self.train_labels = train_labels
        self.test_data = test_data
        self.test_labels = test_labels
        self.population = None
        
        
    def create_tree(self):
        tree = Tree(self.max_depth)
        root = Node('function', self.nvars)
        n_nodes = self.grow_tree(root, 0)
        tree.root = root
        tree.n_nodes = n_nodes
        return tree
    
    
    def grow_tree(self, node, current_depth):
        nodes = 1
        if node.node_type == 'function':
            if current_depth < self.max_depth-1:
                node_type = random.choice(['terminal', 'function'])
                left = Node(node_type, self.nvars)
                node.left = left
                nodes += self.grow_tree(left, current_depth+1)

                node_type = random.choice(['terminal', 'function'])
                right = Node(node_type, self.nvars)
                node.right = right
                nodes += self.grow_tree(right, current_depth+1)
            elif current_depth == self.max_depth-1:
                node_type = 'terminal'
                left = Node(node_type, self.nvars)
                node.left = left

                node_type = 'terminal'
                right = Node(node_type, self.nvars)
                node.right = right
                nodes += 2
                
        return nodes
                
                
    def full_tree(self):
        raise NotImplementedError
        
        
    def create_population(self):
        population = []
        for i in range(self.pop_size):
            population.append(self.create_tree())
        return population
    
    
    def evaluate_population(self, mean_Y):
        for p in self.population:
            p.calculate_fitness(self.train_data, self.train_labels, mean_Y)
    
    
    def crossover(self, parent1, parent2):
        son1 = parent1.copy()
        son2 = parent2.copy()
        selection1 = son1.select_node(random.randint(1, son1.n_nodes-1))
        selection2 = son2.select_node(random.randint(1, son2.n_nodes-1))
        if selection1[2] == 'left':
            selection1[1].left = selection2[0]
        else:
            selection1[1].right = selection2[0]
        if selection2[2] == 'left':
            selection2[1].left = selection1[0]
        else:
            selection2[1].right = selection1[0]
        son1.calculate_n_nodes()    
        son2.calculate_n_nodes()    
        return son1, son2
    
    
    def mutate_point(self, individual):
        selection = individual.select_node(random.randint(0, individual.n_nodes-1))[0]
        if selection.node_type == 'function':
            choice = random.choice(['+', '-', '*', '/'])
            while choice == selection.value:
                choice = random.choice(['+', '-', '*', '/'])
            selection.value = choice
        else:
            choice = selection.value
            while choice == selection.value:
                node_class = random.choice(['x'+str(i) for i in range(selection.nvars)] + ['const'])
                if node_class == 'const':
                    choice = random.random()
                    if random.randint(0,1) == 1:
                        choice *= -1
                else:
                    choice = node_class
            selection.value = choice
        return individual
    
            
    def mutate(self, individual):
        selection = individual.select_node(random.randint(1, individual.n_nodes-1))[0]
        node_class = random.choice(['x'+str(i) for i in range(selection.nvars)] + ['const'])
        if node_class == 'const':
            choice = random.random()
            if random.randint(0,1) == 1:
                choice *= -1
        else:
            choice = node_class
        selection.node_type = 'terminal'
        selection.value = choice
        selection.left = None
        selection.right = None
        individual.calculate_n_nodes()   
        return individual
    
    
    def select(self):
        selected = []
        for i in range(self.tournament):
            chosen = random.choice(self.population)
            while chosen in selected:
                chosen = random.choice(self.population)
            selected.append(chosen)
            
        best = selected[0]
        for tree in selected[1:]:
            if tree.fitness < best.fitness:
                best = tree
        
        return best
    
    
    def generate_new_population(self):
        new_pop = []
        n = 0
        while n < self.pop_size:
            parent1 = self.select()
            parent2 = self.select()
            crossover_prob = random.random()
            if crossover_prob <= self.pc:
                son1, son2 = self.crossover(parent1, parent2)
                mutate_prob = random.random()
                if mutate_prob <= pm:
                    son1 = self.mutate(son1)
                mutate_prob = random.random()
                if mutate_prob <= pm:
                    son2 = self.mutate(son2)
                new_pop.append(son1)
                new_pop.append(son2)
            else:
                mutate_prob = random.random()
                if mutate_prob <= pm:
                    parent1 = self.mutate(parent1)
                mutate_prob = random.random()
                if mutate_prob <= pm:
                    parent2 = self.mutate(parent2)
                new_pop.append(parent1)
                new_pop.append(parent2)
            n += 2
            
        return new_pop
    
    
    def best_individual(self):
        best = self.population[0]
        for p in self.population[1:]:
            if p.fitness < best.fitness:
                best = p
        return best
    
    
    def evolve(self):
        mean_Y = np.mean(train_labels)
        self.population = self.create_population()
        self.evaluate_population(mean_Y)
        generation = 1
        print('Generation', generation)
        current_best = self.best_individual()
        print(current_best)
        print(current_best.fitness)
        while generation < self.ngenerations:
            self.population = self.generate_new_population()
            self.evaluate_population(mean_Y)
            generation += 1
            print('\nGeneration', generation)
            current_best = self.best_individual()
            print(current_best)
            print(current_best.fitness)
            
        return self.best_individual()
    
    

In [4]:
dataset_name = 'synth1'
#dataset_name = 'synth2'
#dataset_name = 'concrete'

train_dataset = np.loadtxt("datasets/"+dataset_name+"/"+dataset_name+"-train.csv", delimiter=",")

train_labels = train_dataset[:,-1]
train_data = train_dataset[:,:-1]

test_dataset = np.loadtxt("datasets/"+dataset_name+"/"+dataset_name+"-test.csv", delimiter=",")

test_labels = test_dataset[:,-1]
test_data = test_dataset[:,:-1]

nvars = test_data.shape[1]

In [5]:
pop_size = 100
ngenerations = 100
max_depth = 7
tournament = 2
pc = 0.9
pm = 0.05
elitism = False

mean_Y = np.mean(train_labels)
GP = GeneticProgramming(pop_size, ngenerations, max_depth, tournament, pc, pm, elitism, nvars, train_data, train_labels, test_data, test_labels)
result = GP.evolve()

print(result)
print('\nTrain fitness', result.fitness)
result.calculate_fitness(test_data, test_labels, np.mean(test_labels))
print('Test fitness', result.fitness)


#print('\nTrain')
#for i in range(train_data.shape[0]):
#    print(result.evaluate(train_data[i]), train_labels[i])

print('\nTest')
for i in range(test_data.shape[0]):
    print(result.evaluate(test_data[i]), test_labels[i])

Generation 1
+ -0.8308970641365767 - * x0 x0 * - + - - 0.43256296620510704 x0 x0 x0 / x0 x0 x0
0.8972292854125627

Generation 2
+ -0.8308970641365767 - * x0 x0 * - + - - 0.43256296620510704 x0 x0 x0 / x0 x0 x0
0.8972292854125627

Generation 3
- * / / x1 0.26157426702469333 / x1 x0 x0 x0
0.5951242988093947

Generation 4
- * / / x1 0.26157426702469333 / x1 x0 x0 x0
0.5951242988093947

Generation 5
+ + + + 0.7005777845851892 x0 / * * x1 x0 + / x0 / -0.9384054704625167 x1 * 0.46386483679108936 x1 -0.9510672381640349 x0 / x1 * * 0.9088342211724926 0.942742637880553 0.6999425343615986
0.8085627740809422

Generation 6
+ + + + x0 x0 / * * x1 x0 + / x0 / -0.9384054704625167 x1 * 0.46386483679108936 x1 -0.9510672381640349 x0 / x1 * * 0.9088342211724926 0.942742637880553 0.6999425343615986
0.8465874912065375

Generation 7
+ x1 - * x0 x0 * - + - - 0.43256296620510704 x0 0.7849244542763739 x0 / x0 x0 0.35556378174135506
1.0176802057044703

Generation 8
+ 0.5370958522444993 - * x0 x0 * - + - - 0.432


Generation 49
+ - + * x0 0.6377375513611074 + x0 * -0.10336230988851014 * x0 x1 * - - - * x0 x0 + * x0 0.6377375513611074 x0 x0 x0 x0 - * + - 0.5579901562064773 0.6377375513611074 - + - x0 0.6377375513611074 x0 - + - - * x0 x0 x0 x0 x0 * x0 x0 x0 x0
0.35117731778664746

Generation 50
+ - + * x0 0.6377375513611074 + x0 0.6377375513611074 * - - - * x0 x0 + * x0 0.6377375513611074 x0 x0 x0 x0 - * + - 0.5579901562064773 0.6377375513611074 - + - x0 0.6377375513611074 x0 - + - - * x0 x0 x0 x0 x0 * x0 x0 x0 x0
0.36376833706140305

Generation 51
+ - + * x0 0.6377375513611074 + x0 0.6377375513611074 * - - - * x0 x0 + * x0 0.6377375513611074 x0 x0 x0 x0 - * + - 0.5579901562064773 0.6377375513611074 - + - x0 0.6377375513611074 x0 - + - - * x0 x0 x0 x0 x0 * x0 x0 x0 x0
0.36376833706140305

Generation 52
+ - + * x0 0.6377375513611074 + x0 * -0.10336230988851014 * x0 x1 * - - - * x0 x0 + * x0 0.6377375513611074 x0 x0 x0 x0 - * + - 0.5579901562064773 0.6377375513611074 - + - x0 0.6377375513611074 0.


Generation 78
+ - + * 0.5579901562064773 x0 + x0 - * x0 x0 x0 * - - 0.6377375513611074 x0 x0 x0 - * + * + - 0.5579901562064773 x0 0.6377375513611074 x0 - + - - * x0 x0 * * + + - - - * x0 x0 + * - 0.5579901562064773 x0 0.6377375513611074 x0 x0 x0 x0 x0 * + - 0.5579901562064773 0.6377375513611074 - x0 x0 - * * x0 0.6377375513611074 x0 x0 x0 x0 x0 - + - - x0 x0 x0 x0 x0 x0 x0
0.30523041392515315

Generation 79
+ - + * 0.5579901562064773 x0 - - - * x0 x0 + * - * x0 x0 + * x0 0.6377375513611074 + x0 x0 0.6377375513611074 + x0 x0 0.6377375513611074 x0 * - - 0.6377375513611074 x0 x0 x0 - * + * + - 0.5579901562064773 x0 0.6377375513611074 x0 - + - - * x0 x0 * * + + - - - * x0 x0 + * - 0.5579901562064773 x0 0.6377375513611074 x0 x0 x0 x0 x0 * + - 0.5579901562064773 0.6377375513611074 - x0 x0 - * * x0 0.6377375513611074 x0 x0 x0 x0 x0 - + - - x0 x0 x0 x0 x0 x0 x0
0.18399426504849656

Generation 80
+ - + * 0.5579901562064773 x0 - - - * x0 x0 + * - * x0 x0 + * + - + * * x0 0.6377375513611074 0.63


Generation 93
+ - + * 0.5579901562064773 x0 - - - * x0 x0 + * 0.6377375513611074 0.6377375513611074 + + - + * * x0 x0 * 0.5579901562064773 x0 0.6377375513611074 x0 + x0 * -0.10336230988851014 * x0 x0 x0 0.6377375513611074 x0 * - - 0.6377375513611074 x0 x0 x0 - * + * + - 0.5579901562064773 x0 0.6377375513611074 x0 - + - - * x0 x0 * * + + - - - * x0 x0 + * - 0.5579901562064773 x0 0.6377375513611074 x0 x0 x0 x0 x0 * + - 0.5579901562064773 0.6377375513611074 - x0 x0 - * * x0 0.6377375513611074 x0 x0 x0 x0 x0 - + - - x0 x0 + + - - - - * x0 + - x0 x0 x0 + * - 0.5579901562064773 x0 0.6377375513611074 x0 - * * x0 0.6377375513611074 x0 x0 x0 x0 x0 x0 x0 x0 x0 0.6377375513611074
0.21749166108530696

Generation 94
+ - + * 0.5579901562064773 x0 - - - * x0 x0 + * 0.6377375513611074 0.6377375513611074 + + - + * * x0 x0 * 0.5579901562064773 x0 0.6377375513611074 x0 + x0 * -0.10336230988851014 * x0 x0 x0 0.6377375513611074 x0 * - - 0.6377375513611074 x0 x0 x0 - * + * + - 0.5579901562064773 x0 0.63773